In [1]:
from detectree2.preprocessing.tiling import tile_data
from detectree2.models.outputs import project_to_geojson, stitch_crowns, clean_crowns
from detectree2.models.predict import predict_on_data
from detectree2.models.train import setup_cfg
from detectron2.engine import DefaultPredictor
import rasterio

In [2]:
# Path to site folder and orthomosaic
site_path = "/mnt/d_drive/detectree2/griffith_site"
img_path = site_path + "/rgb/2016_Griffith_visual_cropped_410074_6211212_40_30_7855.tif"
tiles_path = site_path + "/tilespred/"

# Location of trained model
model_path = "/mnt/d_drive/detectree2/griffith_site/models/230103_randresize_full.pth"

# Specify tiling
buffer = 30
tile_width = 40
tile_height = 40
tile_data(img_path, tiles_path, buffer, tile_width, tile_height, dtype_bool = True)

INFO:detectree2.preprocessing.tiling:Tiling complete


In [3]:
trained_model = "/mnt/d_drive/detectree2/griffith_site/models/230103_randresize_full.pth"
cfg = setup_cfg(update_model=trained_model)
predict_on_data(tiles_path, predictor=DefaultPredictor(cfg))

INFO:detectron2.checkpoint.detection_checkpoint:[DetectionCheckpointer] Loading from /mnt/d_drive/detectree2/griffith_site/models/230103_randresize_full.pth ...
INFO:fvcore.common.checkpoint:[Checkpointer] Loading from /mnt/d_drive/detectree2/griffith_site/models/230103_randresize_full.pth ...
/home/ubuntu/miniconda3/envs/pytorch_env/lib/python3.12/site-packages/fvcore/common/checkpoint.py:252: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allow

Predicting 1 files


/home/ubuntu/miniconda3/envs/pytorch_env/lib/python3.12/site-packages/torch/functional.py:534: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /opt/conda/conda-bld/pytorch_1729647406761/work/aten/src/ATen/native/TensorShape.cpp:3595.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


In [4]:
project_to_geojson(tiles_path, tiles_path + "predictions/", tiles_path + "predictions_geo/")

Projecting 1 files


In [5]:
crowns = stitch_crowns(tiles_path + "predictions_geo/", 1)
clean = clean_crowns(crowns, 0.6, confidence=0)

Cleaning 52 crowns
0 / 52 crowns cleaned


In [6]:
clean.to_file(site_path + "/crowns_out.gpkg")

INFO:pyogrio._io:Created 45 records


In [3]:
from utils import scp_to_ec2
source=r"C:\Users\RoozbehKoochak\Downloads\2016_Griffith_visual_cropped_410074_6211212_40_30_7855.tif"
dest='/mnt/d_drive/detectree2/griffith_site/rgb'
scp_to_ec2('3.104.220.22',source=source,destination=dest)

'scp -i "C:\\Users\\RoozbehKoochak\\.ssh\\Rooz_server_key.pem" "C:\\Users\\RoozbehKoochak\\Downloads\\2016_Griffith_visual_cropped_410074_6211212_40_30_7855.tif" ubuntu@3.104.220.22:"/mnt/d_drive/detectree2/griffith_site/rgb"'

In [10]:
from utils import scp_to_local
source='/mnt/d_drive/detectree2/griffith_site/crowns_out.gpkg'
dest=r"C:\Users\RoozbehKoochak\Downloads"
scp_to_ec2('3.104.220.22',source=source,destination=dest)

'scp -i "C:\\Users\\RoozbehKoochak\\.ssh\\Rooz_server_key.pem" "/mnt/d_drive/detectree2/griffith_site/crowns_out.gpkg" ubuntu@3.104.220.22:"C:\\Users\\RoozbehKoochak\\Downloads"'